In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
tqdm.pandas()
keys = ['model_cls', 'task_name', 'checkpoint', 'seed']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# r = analyze_utils.load_results_and_cache_autoprompt_json(save_dir, save_file='r.pkl', one_row_only=True)
r = pd.read_pickle(oj(save_dir, 'r.pkl'))

In [7]:
r.shape

(1587, 63)

In [4]:
# # subselect the data
r = r[r['n_shots'] == 5]
r = r[r['num_learned_tokens'] == 6]
r = r[~(r['model_cls'] == 'suffix')] # original suffix had a reranking step, correct one is just called "suff"

# # add more keys
keys_derived = ['task_collection', 'top_prompt_correctness', 'bleu_top_prompt']
r['task_collection'] = r.task_name.apply(analyze_utils.task_collection)
r['top_prompt_correctness'] = r['reciprocal_rank'] == 1
r['gt_prompt'] = r['task_name'].apply(lambda x: iprompt.data.TASKS[x]['description'])
r['bleu_top_prompt'] = r.progress_apply(lambda x: sentence_bleu(x['gt_prompt'].split(), x['prefixes'].split()), axis=1) # maybe need to split before calling?

d = r[keys + keys_extra + keys_out + keys_derived]
d.to_pickle(oj(save_dir, 'd.pkl'))
d  = pd.read_pickle(oj(save_dir, 'd.pkl'))

  0%|          | 0/1356 [00:00<?, ?it/s]/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Co

In [3]:
d  = pd.read_pickle(oj(save_dir, 'd.pkl'))

# Metrics table

In [7]:
metrics = d.groupby(by=['model_cls', 'task_collection', 'checkpoint']).mean(numeric_only=True).reset_index()
ms = []
# with pd.option_context('display.float_format', lambda x: '%.1e' % x):
with pd.option_context('display.float_format', lambda x: '%.2g' % x):
    for ckpt in metrics.checkpoint.unique():
        m = (
            metrics[metrics.checkpoint == ckpt]
            .pivot_table(index='model_cls', columns='task_collection', values=['reciprocal_rank', 'top_prompt_correctness', 'bleu_top_prompt'])
            .transpose()
        )
        # display(m)
        ms.append(m)

In [ ]:
for m in ms:
    # put iprompt column first
    col = m['iprompt']
    m = m.drop(columns='iprompt')
    m.insert(loc=0, column='iprompt', value=col)

    m_s = (
        m
        .applymap(lambda x: str.format("{:0.2g}", x))
        .apply(lambda x:
               [f"\\textbf{{{i}}}" if x.name ==
                'iprompt' else i for i in x],
               axis=0)
        .style.to_latex(hrules=True)
        .replace(' & task_collection &  &  &  \\\\\n', '')
        .replace(' & model_cls & iprompt & autoprompt & suff',  ' & & iPrompt & AutoPrompt & Suffix')
    )
    rename = {
        'reciprocal_rank': 'MRR',
        'top_prompt_correctness': 'Correctness',
        'bleu_top_prompt': 'BLEU',
    }
    for k in rename.keys():
        s = '\multirow[c]{4}{*}{' + k + '}'
        # print(s)
        m_s = m_s.replace(
            '\multirow[c]{4}{*}{' + k + '}',
            '\parbox[c]{0.3mm}{\multirow{4}{*}{\\rotatebox[origin = c]{90} {' + rename[k] + '}}}'
        )

    print(m_s)

# Prompts table

In [8]:
for ckpt in d.checkpoint.unique():
    print(ckpt)
    prompts = d[d.checkpoint == ckpt]
    print(prompts.shape[0])
    prompts = (
        prompts
        .sort_values(by=['task_collection', 'reciprocal_rank'], ascending=False)
        .groupby(by=['model_cls', 'task_name'])
        .first()
    )
    # with pd.option_context('display.float_format', lambda x: '%.1e' % x):
    with pd.option_context('display.max_rows', None):
        display(prompts[['prefixes']].pivot_table(index='task_name', columns='model_cls', values='prefixes', aggfunc=lambda x: ' '.join(x)))

EleutherAI/gpt-j-6B
808


model_cls,autoprompt,iprompt,suff
task_name,,,
active_to_passive,(= 18 the the subst,Choose a pronoun for each sentence,Create a sentence or group of
add_two,>:Returns Adding togetherFont accomplish,Create a function named `sum,n>2 m1
antonyms,čthe bectheBut But,The noun to its opposite (,NaN
cause_and_effect,REG Kinect virginity developed mosquit The,What would each sentence be if,write programs that read through an
common_concept,???????? parted configuredthe ????????,Find a noun that includes all,\r __A doctor is
d3_0,NaN,line contains this string? No,string was in one of following
d3_1,NaN,"sentence contains ""yes"" or",string doesn't match any template
d3_10,NaN,"message contains ""no"". No",NaN
d3_11,NaN,"string contains ""No"" or",NaN


google/flan-t5-xl
224


model_cls,iprompt,suff
task_name,,
active_to_passive,Answer: The judge encouraged the,NaN
add_two,prompt: find the sum of,Answer: b. To
antonyms,What is the opposite of what,NaN
cause_and_effect,Answer: Mary's.,NaN
common_concept,answer: sewing wrenches,NaN
d3_0,NaN,Answer: yes publique à
d3_10,Answer: no.||,NaN
d3_11,NaN,Answer: it is not possible
d3_14,Answer: yes or no?,NaN


facebook/galactica-6.7b
324


model_cls,autoprompt,iprompt,suff
task_name,,,
active_to_passive,NaN,4-way Multiple Choice,NaN
add_two,ě addthe Hyper� addi,In order to add two or,NaN
antonyms,meet equilibration stiptertead asymmetry,What is the opposite of each,NaN
cause_and_effect,shaking Dthethethethe,Find clues as to why each,NaN
common_concept,NaN,Where are all the animals?,/en#41
d3_0,NaN,NaN,says that certain issues have received
d3_17,NaN,data includes at least one Sem,NaN
d3_19,NaN,NaN,"text contains the word """
d3_22,NaN,no. For ``yes,NaN
